# Calculate generation capacity by month
This notebook uses the december 2017 EIA-860m file to determine operable generating capacity by fuel category in every month from 2001-2017.

## Instructions
The most recent EIA-860m file should be downloaded to the `EIA downloads` folder, and the correct file name should be used for loading data. Otherwise the code below can be run straight through as-is.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import pathlib
from pathlib import Path
import sys
from os.path import join
import json
import calendar
sns.set(style='white')

idx = pd.IndexSlice

In [2]:
%load_ext watermark
%watermark -iv -v

seaborn     0.8.1
pandas      0.22.0
json        2.0.9
CPython 3.6.4
IPython 6.2.1


In [3]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [5]:
# add the 'src' directory as one where we can import modules
cwd = os.getcwd()
src_dir = Path(cwd, os.pardir, 'src')
sys.path.append(src_dir)

data_path = Path(cwd, '..', 'Data storage')

In [6]:
%aimport Analysis.index
from Analysis.index import group_fuel_cats

## Load monthly EIA-860 data

In [7]:
file_path = data_path / 'EIA downloads' / 'december_generator2017.xlsx'
op = pd.read_excel(file_path, sheet_name='Operating', skiprows=1, skip_footer=1,
                   parse_dates={'op datetime': [14, 15]},
                   na_values=' ')

In [8]:
op.head()

,op datetime,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Unit Code,Nameplate Capacity (MW),...,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude,Google Map,Bing Map,Balancing Authority Code
0,1963-07-01,195,Alabama Power Co,2,Bankhead Dam,Electric Utility,AL,1,NaN,53.9,...,NaN,NaN,NaN,NaN,Tuscaloosa,33.458665,-87.35682,Map,Map,SOCO
1,1954-02-01,195,Alabama Power Co,3,Barry,Electric Utility,AL,1,NaN,153.1,...,NaN,NaN,NaN,NaN,Mobile,31.006900,-88.01030,Map,Map,SOCO
2,1954-07-01,195,Alabama Power Co,3,Barry,Electric Utility,AL,2,NaN,153.1,...,NaN,NaN,NaN,NaN,Mobile,31.006900,-88.01030,Map,Map,SOCO
3,1969-12-01,195,Alabama Power Co,3,Barry,Electric Utility,AL,4,NaN,403.7,...,NaN,NaN,NaN,NaN,Mobile,31.006900,-88.01030,Map,Map,SOCO
4,1971-10-01,195,Alabama Power Co,3,Barry,Electric Utility,AL,5,NaN,788.8,...,NaN,NaN,NaN,NaN,Mobile,31.006900,-88.01030,Map,Map,SOCO


In [9]:
# need to make some helper functions for the retired generators sheet
def bad_month_values(month):
    'Change value to 1 if outside 1-12'

    if month > 12 or month < 1:
        new_month = 1
    else:
        new_month = month
    return new_month

def make_dt_col(df, month_col, year_col):
    months = df[month_col].astype(str)
    years = df[year_col].astype(str)
    dt_string = years + '-' + months + '-' + '01'
    dt = pd.to_datetime(dt_string)
    return dt

ret = pd.read_excel(file_path, sheet_name='Retired', skiprows=1, skip_footer=1,
                    converters={'Operating Month': bad_month_values},
                    # parse_dates={'op datetime': [16, 17],
                    #              'ret datetime': [14, 15]},
                    na_values=' ')

ret['op datetime'] = make_dt_col(ret, 'Operating Month', 'Operating Year')
ret['ret datetime'] = make_dt_col(ret, 'Retirement Month', 'Retirement Year')

## Clean up column names and only keep desired columns

In [10]:
op.columns = op.columns.str.strip()
ret.columns = ret.columns.str.strip()

In [11]:
op_cols = [
    'Plant ID', 'Nameplate Capacity (MW)', 'Net Summer Capacity (MW)',
    'Energy Source Code', 'Prime Mover Code', 'op datetime'
]

ret_cols = [
    'Plant ID', 'Nameplate Capacity (MW)', 'Net Summer Capacity (MW)',
    'Energy Source Code', 'Prime Mover Code', 'Retirement Month',
    'Retirement Year', 'Operating Month', 'Operating Year',
    'op datetime', 'ret datetime'
]

In [12]:
op = op.loc[:, op_cols]
ret = ret.loc[:, ret_cols]

op.columns = op.columns.str.lower()
ret.columns = ret.columns.str.lower()

In [13]:
op.head()

,plant id,nameplate capacity (mw),net summer capacity (mw),energy source code,prime mover code,op datetime
0,2,53.9,56.0,WAT,HY,1963-07-01
1,3,153.1,55.0,NG,ST,1954-02-01
2,3,153.1,55.0,NG,ST,1954-07-01
3,3,403.7,362.0,BIT,ST,1969-12-01
4,3,788.8,738.5,BIT,ST,1971-10-01


## Read fuel category definitions and apply to the generators

In [14]:
state_cat_path = data_path / 'Fuel categories' / 'State_facility.json'
custom_cat_path = data_path / 'Fuel categories' / 'Custom_results.json'
with open(state_cat_path) as json_data:
    state_cats = json.load(json_data)
with open(custom_cat_path) as json_data:
    custom_cats = json.load(json_data)

In [15]:
def reverse_cats(cat_file):
    'Reverse a dict of lists so each item in the list is a key'
    cat_map = {}
    for key, vals in cat_file.items():
        for val in vals:
            cat_map[val] = key
    return cat_map

In [16]:
# Aggregate EIA fuel codes to my final definitions
op['fuel'] = op.loc[:, 'energy source code'].map(reverse_cats(state_cats))
op['fuel category'] = op.loc[:, 'fuel'].map(reverse_cats(custom_cats))

ret['fuel'] = ret.loc[:, 'energy source code'].map(reverse_cats(state_cats))
ret['fuel category'] = ret.loc[:, 'fuel'].map(reverse_cats(custom_cats))

In [17]:
op.head()

,plant id,nameplate capacity (mw),net summer capacity (mw),energy source code,prime mover code,op datetime,fuel,fuel category
0,2,53.9,56.0,WAT,HY,1963-07-01,HYC,Hydro
1,3,153.1,55.0,NG,ST,1954-02-01,NG,Natural Gas
2,3,153.1,55.0,NG,ST,1954-07-01,NG,Natural Gas
3,3,403.7,362.0,BIT,ST,1969-12-01,COW,Coal
4,3,788.8,738.5,BIT,ST,1971-10-01,COW,Coal


## Load the NERC region each power plant is in and add to dataframes

In [18]:
nercs_path = data_path / 'Facility labels' / 'Facility locations_knn.csv'
facility_nerc = pd.read_csv(nercs_path)

In [19]:
# Add NERC regions to each generator
op = op.merge(facility_nerc.loc[:, ['plant id', 'nerc']], on='plant id')
ret = ret.merge(facility_nerc.loc[:, ['plant id', 'nerc']], on='plant id')

## Determine operable capacity in every month

In [20]:
# Define iterables to loop over
years = range(2001,2018)
months = range(1,13)
nercs = op['nerc'].dropna().unique()
fuels = op['fuel category'].dropna().unique()

# Create a dataframe with the nerc/fuel/year/month index levels
index = pd.MultiIndex.from_product([nercs, fuels, years, months],
                                   names=['nerc', 'fuel category', 'year', 'month'])
op_df_capacity = pd.DataFrame(index=index, columns=['active capacity', 'possible gen'])
op_df_capacity.sort_index(inplace=True)

In [21]:
def month_hours(year, month):
    days = calendar.monthrange(year, month)[-1]
    return days * 24

In [55]:
# This is slow but it works.
# Find the active and retired capacity for every month
for year in years:
    print(year)

    for month in months:
        dt = pd.to_datetime('{}-{}-01'.format(year, month), yearfirst=True)

        for nerc in nercs:
            # op_dict[dt][nerc] = {}
            for fuel in fuels:

                plants_op = (op.loc[(op['op datetime'] <= dt) &
                                    (op['nerc'] == nerc) &
                                    (op['fuel category'] == fuel),
                                    'nameplate capacity (mw)']
                                # .dropna()
                                .sum())

                retired = (ret.loc[(ret['ret datetime'] >= dt) &
                                   (ret['op datetime'] <= dt) &
                                   (ret['nerc'] == nerc) &
                                   (ret['fuel category'] == fuel),
                                   'nameplate capacity (mw)']
                              # .dropna()
                              .sum())

                op_df_capacity.loc[idx[nerc, fuel, year, month], 'active capacity'] = plants_op + retired
                op_df_capacity.loc[idx[nerc, fuel, year, month], 'possible gen'] = month_hours(year, month) * (plants_op + retired)

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


In [56]:
# Add datetime from the year and month
op_df_capacity['datetime'] = (pd.to_datetime(
    op_df_capacity.index
                  .get_level_values('year')
                  .astype(str)
    + '-'
    + op_df_capacity.index
                    .get_level_values('month')
                    .astype(str)
    + '-01'))

In [35]:
# Write data to file
out_path = data_path / 'Derived data' / 'Plant capacity' / 'monthly capacity by fuel.csv'
op_df_capacity.to_csv(out_path)

## Determine natural gas capacity by prime mover type in each month

In [22]:
# Create an empty dataframe
index = pd.MultiIndex.from_product([nercs, years, months],
                                   names=['nerc', 'year', 'month'])
op_ng_type = pd.DataFrame(index=index,
                          columns=['ngcc', 'turbine', 'other', 'total',
                                   'ngcc fraction', 'turbine fraction',
                                   'other fraction'])
op_ng_type.sort_index(inplace=True)

In [23]:
# Create individual dataframes of each prime mover type
# from the operable and retired dataframes
# "Other" is defined as internal combustion and steam turbine

# Lists of prime mover codes for each category
ngcc_pm = ['CA', 'CS', 'CT']
turbine_pm = ['GT']
other_pm = ['IC', 'ST']


op_ngcc = op.loc[(op['fuel category'] == 'Natural Gas') &
                 (op['prime mover code'].isin(ngcc_pm)), :]
op_turbine = op.loc[(op['fuel category'] == 'Natural Gas') &
                 (op['prime mover code'].isin(turbine_pm)), :]
op_other = op.loc[(op['fuel category'] == 'Natural Gas') &
                  (op['prime mover code'].isin(other_pm)), :]

ret_ngcc = ret.loc[(ret['fuel category'] == 'Natural Gas') &
                 (ret['prime mover code'].isin(ngcc_pm)), :]
ret_turbine = ret.loc[(ret['fuel category'] == 'Natural Gas') &
                  (ret['prime mover code'].isin(turbine_pm)), :]
ret_other = ret.loc[(ret['fuel category'] == 'Natural Gas') &
                  (ret['prime mover code'].isin(other_pm)), :]

In [24]:
for year in years:
    print(year)

    for month in months:
        dt = pd.to_datetime('{}-{}-01'.format(year, month), yearfirst=True)

        for nerc in nercs:

            ngcc = (
                op_ngcc.loc[(op_ngcc['op datetime'] <= dt) &
                                    (op_ngcc['nerc'] == nerc),
                                    'nameplate capacity (mw)'].sum()
                + ret_ngcc.loc[(ret_ngcc['ret datetime'] >= dt) &
                                    (ret_ngcc['op datetime'] <= dt) &
                                    (ret_ngcc['nerc'] == nerc),
                                    'nameplate capacity (mw)'].sum()
            )

            turbine = (
                op_turbine.loc[(op_turbine['op datetime'] <= dt) &
                                    (op_turbine['nerc'] == nerc),
                                    'nameplate capacity (mw)'].sum()
                + ret_turbine.loc[(ret_turbine['ret datetime'] >= dt) &
                                    (ret_turbine['op datetime'] <= dt) &
                                    (ret_turbine['nerc'] == nerc),
                                    'nameplate capacity (mw)'].sum()
            )

            other = (
                op_other.loc[(op_other['op datetime'] <= dt) &
                                    (op_other['nerc'] == nerc),
                                    'nameplate capacity (mw)'].sum()
                + ret_other.loc[(ret_other['ret datetime'] >= dt) &
                                    (ret_other['op datetime'] <= dt) &
                                    (ret_other['nerc'] == nerc),
                                    'nameplate capacity (mw)'].sum()
            )
            total = ngcc + turbine + other

            op_ng_type.loc[idx[nerc, year, month], 'total'] = total
            op_ng_type.loc[idx[nerc, year, month], 'ngcc'] = ngcc
            op_ng_type.loc[idx[nerc, year, month], 'turbine'] = turbine
            op_ng_type.loc[idx[nerc, year, month], 'other'] = other

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


In [25]:
op_ng_type['ngcc fraction'] = op_ng_type['ngcc'] / op_ng_type['total']
op_ng_type['turbine fraction'] = op_ng_type['turbine'] / op_ng_type['total']
op_ng_type['other fraction'] = op_ng_type['other'] / op_ng_type['total']
op_ng_type.fillna(0, inplace=True)

op_ng_type['datetime'] = (pd.to_datetime(
    op_ng_type.index
                  .get_level_values('year')
                  .astype(str)
    + '-'
    + op_ng_type.index
                    .get_level_values('month')
                    .astype(str)
    + '-01'))

In [32]:
out_path = data_path / 'Derived data' / 'Plant capacity' / 'monthly natural gas split.csv'
op_ng_type.to_csv(out_path)